In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['GDAL_SKIP'] = 'DODS'

from geostacks import SpatialIndexLS8, SpatialIndexITSLIVE

import logging
import isce
root_logger = logging.getLogger()
root_logger.setLevel('WARNING')

from carst import SingleRaster, RasterVelos, ConfParams
from carst.libft import ampcor_task, writeout_ampcor_task
from carst.libxyz import ZArray, DuoZArray, AmpcoroffFile, points_in_polygon

import ipyleaflet as ilfl
import ipywidgets as iwg

import pandas as pd

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce


In [4]:
ls8_index = SpatialIndexLS8('./LS8_cornerPts.xlsx')
ls8_index.read()
# ls8_index.footprint
query_pt = [-50., 69.]
idxs = ls8_index.query_pathrow(query_pt)
ls8_index.footprint.loc[idxs]

,path,row,lat_CTR,lon_CTR,geometry
1747,8,12,68.279631,-49.545762,"POLYGON ((-50.91000 69.35600, -46.41400 68.660..."
1994,9,11,69.606373,-49.402408,"POLYGON ((-50.77700 70.69500, -46.05000 69.955..."
2242,10,11,69.606373,-50.947472,"POLYGON ((-52.32200 70.69500, -47.59500 69.955..."
20071,81,233,69.606373,-48.123782,"POLYGON ((-44.88600 69.19800, -49.35900 68.507..."
20319,82,233,69.606373,-49.668847,"POLYGON ((-46.43100 69.19800, -50.90400 68.507..."
20566,83,232,68.279631,-49.525493,"POLYGON ((-46.50100 67.84400, -50.76100 67.192..."
20567,83,233,69.606373,-51.213911,"POLYGON ((-47.97600 69.19800, -52.44900 68.507..."
20814,84,232,68.279631,-51.070557,"POLYGON ((-48.04600 67.84400, -52.30600 67.192..."


In [5]:
def carst_featuretrack(file1_url, file1_date, file2_url, file2_date, inipath='param.ini'):
    ini = ConfParams(inipath)
    ini.ReadParam()
    ini.imagepair['image1'] = file1_url
    ini.imagepair['image2'] = file2_url
    ini.imagepair['image1_date'] = file1_date
    ini.imagepair['image2_date'] = file2_date
    ini.VerifyParam()
    a = SingleRaster(file1_url, date=file1_date)
    b = SingleRaster(file2_url, date=file2_date)
    if ini.pxsettings['gaussian_hp']:
        a.GaussianHighPass(sigma=ini.pxsettings['gaussian_hp_sigma'])
        b.GaussianHighPass(sigma=ini.pxsettings['gaussian_hp_sigma'])
    a.AmpcorPrep()
    b.AmpcorPrep()
    task = ampcor_task([a, b], ini)
    writeout_ampcor_task(task, ini)
    ampoff = AmpcoroffFile(ini.rawoutput['label_ampcor'] + '.p')
    ampoff.Load()
    ampoff.SetIni(ini)
    ampoff.FillwithNAN()   # fill holes with nan
    ampoff.Ampcoroff2Velo()
    ampoff.Velo2XYV(generate_xyztext=ini.rawoutput['if_generate_xyztext'])
    ampoff.XYV2Raster()

In [6]:
# # Adapted from Alice's AGU talk
# zoom = 4
# query_pt = [-50., 69.]
# mainmap = ilfl.Map(basemap=ilfl.basemaps.Gaode.Satellite, center=[query_pt[-1], query_pt[0]], zoom=zoom)

# mker = ilfl.Marker(location=[query_pt[-1], query_pt[0]], draggable=True)
# mainmap.add_layer(mker)

# pr_options = [('{:03d}/{:03d}'.format(ls8_index.footprint.loc[i, 'path'], ls8_index.footprint.loc[i, 'row']) ,i) for i in idx]
# pr_menu = iwg.Select(options=pr_options, description='Path/Row:', rows=15)
# pr_scene_options = []
# pr_scene_menu = iwg.SelectMultiple(options=pr_scene_options, description='Data entries:', rows=20)
# pr_kernel_selection = iwg.RadioButtons(options=['ITS_LIVE (online ready)', 'CARST'], value='ITS_LIVE (online ready)', description='Data / Kernel:')
# pr_scene_button = iwg.Button(description='Search for dates')


# band_radiobuttons = iwg.RadioButtons(options=['B4', 'B8'], value='B8', description='Band:')
# ft_start_button = iwg.Button(description='Start feature tracking')

# pr_selection = idx[0]
# pr_scene_list = pd.DataFrame(columns=('prefix', 'time', 'tier'))
# pr_polygon = ilfl.WKTLayer(wkt_string=ls8_index.footprint.loc[pr_selection].geometry.wkt)
# mainmap.add_layer(pr_polygon)

# output = iwg.Output()

# # ==== map marker callback

# def on_location_changed(event):
#     global query_pt, idxs
#     query_pt = [mker.location[-1], mker.location[0]]
#     idxs = ls8_index.query_pathrow(query_pt)
#     pr_options_new = [('{:03d}/{:03d}'.format(ls8_index.footprint.loc[i, 'path'], ls8_index.footprint.loc[i, 'row']) ,i) for i in idxs]
#     pr_menu.options = pr_options_new
#     with output:
#         print(idxs)

# mker.observe(on_location_changed, 'location')

# # ==== left side menu selection callback

# def on_menu_selection_changed(change):
#     global pr_selection
#     pr_selection = change['new']
#     pr_polygon.wkt_string=ls8_index.footprint.loc[pr_selection].geometry.wkt

# pr_menu.observe(on_menu_selection_changed, names='value')

# # ==== search button click callback

# def on_searchbutton_clicked(b):
#     global pr_scene_list
#     with output:
#         if pr_kernel_selection.value == 'CARST':
#             s3_prefix, pr_scene_list = ls8_index.search_s3(pr_selection)
#             # print(s3_prefix)
#             pr_scene_menu.options = [(record['time'], idx) for idx, record in pr_scene_list.loc[pr_scene_list['tier'] == 'T1'].iterrows()]
#         elif pr_kernel_selection.value == 'ITS_LIVE (online ready)':
#             query_pt = [mker.location[-1], mker.location[0]]
#             polygon_coords = itslive_api.get_minimal_bbox(query_pt)
#             params = {'polygon': polygon_coords, 'percent_valid_pixels': 1, 'start': '2015-01-01', 'end': '2020-01-01'}
#             urls = itslive_api.get_granule_urls(params)
#             pr_dict = itslive_api.parse_urls(urls)
#             pr_dict_key = '{:03d}/{:03d}'.format(ls8_gdf.loc[pr_selection, 'path'], ls8_gdf.loc[pr_selection, 'row'])
#             pr_scene_menu.options = pr_dict[pr_dict_key]
            
# pr_scene_button.on_click(on_searchbutton_clicked)

# # ==== feature tracking button click callback

# file1_url  = None 
# file1_date = None 
# file2_url  = None 
# file2_date = None

# def on_ftbutton_clicked(ft):
#     global file1_url, file1_date, file2_url, file2_date
#     with output:
#         selected_list = pr_scene_list.loc[list(pr_scene_menu.value)]
#         selected_list_prefix = selected_list['prefix'].tolist()
#         selected_list_time = selected_list['time'].tolist()
#         file1_url = 'https://landsat-pds.s3.amazonaws.com/' + selected_list_prefix[0] + os.path.basename(selected_list_prefix[0][:-1]) + '_' + band_radiobuttons.value + '.TIF'
#         file2_url = 'https://landsat-pds.s3.amazonaws.com/' + selected_list_prefix[1] + os.path.basename(selected_list_prefix[1][:-1]) + '_' + band_radiobuttons.value + '.TIF'
#         file1_date = selected_list_time[0].strftime('%Y-%m-%d')
#         file2_date = selected_list_time[1].strftime('%Y-%m-%d')
#         print(file1_url, file1_date, file2_url, file2_date)
#         carst_featuretrack(file1_url, file1_date, file2_url, file2_date)

# ft_start_button.on_click(on_ftbutton_clicked)



# leftside = iwg.VBox([iwg.HTML("<h2>Drag the marker to your region of interest</h2>"), pr_menu, pr_kernel_selection, pr_scene_button])
# leftside.layout.align_items = 'center'

# rightside = iwg.VBox([pr_scene_menu, band_radiobuttons, ft_start_button])
# rightside.layout.align_items = 'center'

# iwg.AppLayout(left_sidebar=leftside, center=mainmap, right_sidebar=rightside)

In [22]:
# output

Output()

In [13]:
from eztrack import eztrack_ui

cpanel = eztrack_ui(spatial_index=ls8_index)
cpanel.gen_ui()

AppLayout(children=(VBox(children=(HTML(value='<h2>Drag the marker to your region of interest</h2>'), Select(d…